In [3]:
import pandas as pd
import numpy as np
import glob
import os

In [32]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

## 1- Clean kaggle data (only US and France):

In [4]:
kaggle_tweets = pd.read_csv('../data/tweets_us/kaggle_en_tweets/vaccination_all_tweets.csv')

In [5]:
kaggle_tweets.shape

(69718, 16)

In [8]:
kaggle_tweets.head(2)

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet
0,1340539111971516416,Rachel Roh,"La Crescenta-Montrose, CA","Aggregator of Asian American news; scanning diverse sources 24/7/365. RT's, Follows and 'Likes' will fuel me 👩‍💻",2009-04-08 17:52:46,405,1692,3247,False,2020-12-20 06:06:44,Same folks said daikon paste could treat a cytokine storm #PfizerBioNTech https://t.co/xeHhIMg1kF,['PfizerBioNTech'],Twitter for Android,0,0,False
1,1338158543359250433,Albert Fong,"San Francisco, CA","Marketing dude, tech geek, heavy metal & '80s music junkie. Fascinated by meteorology and all things in the cloud. Opinions are my own.",2009-09-21 15:27:30,834,666,178,False,2020-12-13 16:27:13,"While the world has been on the wrong side of history this year, hopefully, the biggest vaccination effort we've ev… https://t.co/dlCHrZjkhm",NaN,Twitter Web App,1,1,False


In [49]:
kaggle_tweets = kaggle_tweets.dropna(subset = ['user_location'])

In [50]:
# Get a list of user location (higher count on top)  
#list_cities = kaggle_tweets.groupby('user_location')['user_location'].count().sort_values(ascending=False).to_frame()

### Rename and drop columns:

In [51]:
kaggle_tweets = (kaggle_tweets
          .drop(columns = ['user_created', 'user_favourites', 'source', 'favorites', 'is_retweet'])
          .rename(columns = {'id': 'tweet_id'})
         )


### Keep only tweets in France and USA:

In [35]:
cities_fr = ['France', 'Paris, France', 'Nancy, France', 'Dammam, KSA | Tunisia | France', 'Alsace, France', 'Paris', 'La Belle Province', 'Bordeaux, France', 'Alpes-Maritimes, Provence-Alpe', '#CoronavirusMadeInPasteur', 'Rome, Italy - Beaulieu, France', 'Europe (usually Poland/France)', 'Paris, Limoges. France']
cities_us_north = ['La Crescenta-Montrose, CA', 'San Francisco, CA', 'Los Angeles, CA', 'Rochester, NY', 'Cranbury, NJ', 'USA🇺🇸', 'America', 'Medford, OR', 'LI, NYC, NJ, PA, NE, USA', 'Chicago, IL', 'Seattle, WA',  'New Jersey, USA', 'Oregon, USA', 'California, USA', 'New York, NY', 'Cambridge, MA', 'New Orleans, LA', 'Michigan', 'Philadelphia, Pa.', 'North America', 'Troy, Michigan;  Serial ✈️', 'Pennsylvania, USA', 'Saugerties, New York', 'Balboa Peninsula, SoCal USA', 'New York / New Jersey / Global', 'Minneapolis, MN', 'Vermont, US', 'Naperville, IL', 'Sacramento', 'New York, USA', 'Cherry Hill, NJ', 'NYC', 'San Jose, CA (USA)', 'Colorado Springs, Colorado ', 'Silicon Valley, CA', 'Washington, DC', 'D.C By Way Of BOSTON!🇳🇬', 'New York', 'Mequon, Wisconsin', 'Boston, MA', 'New York City, Westport, CT', 'Michigan, USA', 'US - IL - ID', 'Washington DC', 'CHICAGO!', 'NY', 'Brooklyn, NY', 'Los Angeles', 'Providence, RI', 'Boston', 'New York City', 'Rancho Cucamonga, CA', 'Syracuse, NY', 'New York & San Francisco', 'Philadelphia, PA', 'New Jersey', 'Buffalo, NY', 'Connecticut, USA', 'Rhode Island, USA', 'Northern Illinois', 'Rochester NY', 'Southern CA', 'Staten Island, New York', 'South Lake Tahoe, CA', 'Boston, MA.', 'washington', 'Paramus, NJ', 'Orange County, CA', 'Elmhurst, IL', 'Grand Rapids, MI', 'Long Beach, CA', 'Washington, D.C.', 'Northern California, USA', 'Princeton, NJ', 'Ventura, CA ', 'Washington D.C.', 'Rochester, MN', 'Denver, CO', 'Englewood, NJ', 'Katonah, NY', 'Chicago', 'Valencia,CA', 'Bellevue, WA', 'Minnesota, USA', 'Old Mystic, CT USA', 'San Francisco Bay Area', 'Santa Barbara, CA', 'Baltimore', 'Vermont, USA', 'Sunnyvale, CA', 'Bethesda, MD.', 'Queens, NY', 'Rochester MN', 'Social distancing in Manhattan', 'Morristown, NJ', 'Bronx, NY', 'DC Metro Area, USA', 'Maryland, USA', 'Stony Brook,  NY', 'Bryn Mawr, PA', 'Madison, WI', 'Coventry Rhode Island', 'Ann Arbor, ✋', 'Colorado, USA', 'Baltimore, MD', 'Pittsburgh, PA', 'Wisconsin, USA', 'minneapolis', 'Southern California', 'Northern California', 'Potomac  MD', 'Seaside, California, USA', 'Hartford, CT', 'Long Island   ', 'California', 'Paterson, NJ', 'NH', 'Manhattan, NYC', 'California Bay Area', 'VT', 'Stanford, CA', 'Duarte, CA', 'Los Angeles, California', 'Central California', 'Portland, ME', 'Philadelphia', 'Riverdale Park, MD ', 'Manhattan, NY', 'East Hampton, NY', 'Hollywood, CA, USA', 'South Cal', 'Ann Arbor, MI', 'Batavia, IL', 'Cambridge, MA, United States', 'Chicago-adjacent, IL', 'Metro North of Boston MA', 'New Haven, CT', 'Silver Spring, MD', 'West Hollywood, CA', 'So Cal', 'SoCal', 'California, United States', 'Sacramento, CA', 'Massachusetts, USA', 'Illinois, USA', 'Washington, USA', 'Fremont, CA, USA', 'New Jersey', 'Portland, OR', 'San Francisco', 'New York, New York', 'Santa Monica, CA', 'Connecticut', 'West Hartford, CT', 'Oakland, CA', 'Anaheim, CA', 'San Jose, CA', 'Buffalo, NY', 'Massachusetts', 'Los Angeles, California', 'Vermont, USA', 'Detroit, MI', 'Delaware, USA', 'Philadelphia', 'Boothbay Harbor, Maine', 'San Diego, California']
cities_us_south = ['Texas', 'Jefferson City, MO',  'LA | OC ', 'Beaumont, Texas', 'Kentucky', 'Dallas, TX', 'Lakeland, FL', 'Palm Beach, FL', 'Tampa, FL', 'Okolona, KY', 'Houston, TX', 'Miami, FL', 'McAllen, TX', 'Texas, USA', 'Miami', 'Charleston, SC', 'Florida, USA', 'West Virginia, USA', 'Raleigh, NC', 'Raleigh, NC, USA', 'South Carolina, USA', 'Durham, NC', 'Alabama', 'Cincinnati, OH ', '11150A Asheville Hwy Inman, SC', 'Tucson, AZ', 'Albuquerque, NM', 'Nashville, TN', 'St Petersburg, FL', 'Austin, TX', 'Kansas', 'Roanoke, VA', 'Clinton, SC', 'Miami Beach, FL', 'San Antonio, Texas', 'Atlanta, GA', 'Odessa, TX', 'Chapel Hill, NC', 'Scottsdale, AZ', 'NEVADA', 'Richmond, VA', 'Atlanta, Georgia', 'Tysons Corner, Virginia', 'Phoenix, AZ', 'Greensboro, NC', 'Las Vegas, NV', 'Harlingen, TX', 'North Carolina, USA', 'Indianapolis, IN', 'Spring, TX', 'Idaho, USA', 'Athens, GA', 'Jacksonville, FL', 'Houston Texas Based', 'Jackson, MS', 'Covid Hell, AL', 'Kansas City, KS', 'Fort Worth', 'Charlottesville, VA', 'Kansas City, MO', 'Florida', 'Dayton, OH', 'Saint Louis, MO, USA', 'St Louis, MO', 'Knoxville, TN', 'Birmingham, AL', 'Columbia, MO', 'Morgantown, WV', 'TX', 'Allen, TX', 'Arkansas', 'Gainesville, FL', 'Smithville, TX', 'Augusta, GA', 'West Texas', 'Oklahoma City & Tulsa, OK', 'Oklahoma, USA', 'Gainesville, FL, USA', 'Arkansas, United States', 'Sugar Land, TX', 'Miami, Fl', 'Central FL', 'arkansas', 'Riverdale, UT', 'Nashville', 'Dallas', 'Houston', 'Reno, NV', 'Tampa Bay, FL', 'Charlotte, NC', 'Austin', 'Kansas City ', 'South Florida', 'Salt Lake City, UT', 'Alabama, USA', 'Ohio, USA', 'Virginia, USA', 'Orlando, FL', 'Las Vegas', 'Lufkin, TX', 'Arizona, USA', 'Cleveland, OH', 'Georgia, USA', 'Fort Lauderdale, FL', 'New York, Sarasota, FL', 'Columbus, OH', 'Louisville, KY', 'Tennessee, USA', 'Missouri, USA', 'Indiana, USA', 'Kentucky, USA', 'Oklahoma City, OK', 'North Carolina', 'New Mexico, USA', 'Rio Grande Valley, TX', 'Houston, Tx', 'Ohio', 'Cleveland, Ohio', 'Utah, USA', 'Indiana', 'Salt Lake City, Utah', 'Dallas, Texas', 'South Carolina', 'Columbus, Ohio', 'Corpus Christi, TX', 'Tampa, FL - United States', 'Atlanta', 'Indianapolis', 'Salt Lake City, UT', 'Alabama', 'TX', 'Fort Worth, TX', 'Jacksonville, N.C.', 'Tennessee', 'Arkansas, USA', 'Baton Rouge, LA', 'Arkansas', 'Jupiter, FL', 'Louisiana, USA', 'Tulsa, OK']

In [52]:
tweets_fr = kaggle_tweets[kaggle_tweets['user_location'].isin(cities_fr)]
tweets_us_north = kaggle_tweets[kaggle_tweets['user_location'].isin(cities_us_north)]
tweets_us_south = kaggle_tweets[kaggle_tweets['user_location'].isin(cities_us_south)]

In [53]:
tweets_us_south.head(1)

,tweet_id,user_name,user_location,user_description,user_followers,user_friends,user_verified,date,text,hashtags,retweets
64,1338594342265556995,Emily Manley,"Jefferson City, MO",#MO Chief Capitol reporter for @FOX2now @KPLR11 @fox4kc @KSNLocalNews @kolr10kozl @KODENews - @UISedu PAR & @WesternILUniv Alum - #STLCards & #STLBlues fan,1494,1646,False,2020-12-14 21:18:55,#INBOX | @GovParsonMO says shipments of the #PfizerBioNTech vaccine will continue to ship to Missouri’s 21 vaccinat… https://t.co/w5xqB5gbW3,"['INBOX', 'PfizerBioNTech']",5


## 2- Clean data from Tweepy:

### Merge files to one file:

**France:**

In [58]:
# merging astrazeneca files
astra_fr = pd.concat(
    map(pd.read_csv, ['../data/tweets_fr/astra/tweets_fr_tweets_fr_aztra_2021_05_18_to_05_12.csv', '../data/tweets_fr/astra/tweets_fr_tweets_fr_astra_2021_05_19_to_05_13.csv']), ignore_index=True)
astra_fr.shape

(667, 9)

In [61]:
# Make a new column with vaccine name
astra_fr['vaccine'] = 'astrazeneca'

In [4]:
# merging J&J files
jandj_fr = pd.concat(
    map(pd.read_csv, ['../data/tweets_fr/JandJ/tweets_fr_jandj_2021_05_19_to_05_14.csv', '../data/tweets_fr/JandJ/tweets_fr_jandj_2021_05_18_to_05_11.csv']), ignore_index=True)
jandj_fr.shape

(69, 9)

In [5]:
jandj_fr.head(1)

,Tweet Id,Tweet Date,location,Follower Count,Account Verified,Favorite Count,Retweets,Tweet Text,hashtags
0,1395070529724370944,2021-05-19 17:35:08,"Paris, France",714,False,1,0,"b""En live du meetup #dataprivacy --&gt; la #p...","[{'text': 'dataprivacy', 'indices': [19, 31]},..."


In [99]:
# Make a new column with vaccine name
jandj_fr['vaccine'] = 'johnson'

In [100]:
# merging pfizer files
pfizer_fr = pd.concat(
    map(pd.read_csv, ['../data/tweets_fr/pfizer/tweets_fr_pfizer_2021_05_14_to_05_11.csv', '../data/tweets_fr/pfizer/tweets_fr_pfizer_2021_05_18_to_05_15.csv']), ignore_index=True)
pfizer_fr.shape

(2577, 8)

In [101]:
# Make a new column with vaccine name
pfizer_fr['vaccine'] = 'pfizer'

In [102]:
# moderna
moderna_fr = pd.read_csv('../data/tweets_fr/moderna/tweets_fr_moderna_2021_05_18_to_05_11.csv')
moderna_fr['vaccine'] = 'moderna'

In [104]:
# Concatenate dataframes together
vaccines_fr = pd.concat([astra_fr, jandj_fr, pfizer_fr, moderna_fr], ignore_index=True)

In [106]:
vaccines_fr.shape

(4111, 10)

In [191]:
test2 = vaccines_fr[["Tweet Id", "Tweet Date", "location", "Follower Count", "Account Verified", "Favorite Count", "Retweets", "Tweet Text", "hashtags"]]

In [192]:
test2 = test2.drop_duplicates()

In [193]:
test2.shape

(4105, 9)

In [125]:
vaccines_fr.head(1)

,Tweet Id,Tweet Date,location,Follower Count,Account Verified,Favorite Count,Retweets,Tweet Text,hashtags,vaccine
0,1393495540940300288,2021-05-15 09:16:41,Bordeaux,1133,False,0,0,"b""@simplettebounty Ah ca c'est un autre pb. De toute facon, la limitation, c'est le nombre de doses achet\xc3\xa9es (les graph sur VaccinTracker motre bien qu'a part Aztrazeneca, on consomme tout ce qui nous est livr\xc3\xa9).\nDonc tant que ca produira pas plus...""",[],astrazeneca


In [64]:
# Get tweets with a different search
all_fr = pd.read_csv('../data/tweets_fr/tweets_fr_all_2021_05_19_to_05_11.csv')

In [65]:
all_fr.shape

(5099, 9)

In [189]:
test = all_fr.drop_duplicates()

In [190]:
test.shape

(5099, 9)

In [ ]:
#columns = ["Tweet Id", "Tweet Date", "location", "Follower Count", "Account Verified", "Favorite Count", "Retweets", "Tweet Text", "hashtags"]

In [185]:
common = all_fr.merge(vaccines_fr, how='inner', on=["Tweet Id"]) 

In [186]:
common.shape

(3206, 18)

In [187]:
all_fr_no_com = all_fr[(~all_fr["Tweet Id"].isin(common["Tweet Id"]))]
all_fr_no_com.shape

(1907, 9)

### Replace 'utf-8' characters with french accents:

In [87]:
### encoding real characters
moderna_fr = pd.read_csv('../data/tweets_fr/moderna/tweets_fr_moderna_2021_05_18_to_05_11.csv') #, encoding = 'ASCII')

In [75]:
moderna_fr.head()

,Tweet Id,Tweet Date,location,Follower Count,Account Verified,Favorite Count,Retweets,Tweet Text
0,1394429433273098240,2021-05-17 23:07:38,Paris,42,False,0,0,b'@BFMTV Moderna est quoi alors?'
1,1394425237501063173,2021-05-17 22:50:58,"Valenciennes, France",17,False,1,0,"b""@Sante_Gouv 1ere dose aujourd'hui en moderna pour moi a part douleur a l'\xc3\xa9paule tout vas bien :)"""
2,1394420316458389512,2021-05-17 22:31:25,Clermont Ferrand,7,False,0,0,"b""\xf0\x9f\x92\x89 3 cr\xc3\xa9neaux disponibles\n\xf0\x9f\x93\x85 Aujourd'hui \xc3\xa0 15:40\n\xf0\x9f\x8f\xa5 Centre de vaccination COVID des Combrailles - Pontaumur (Moderna)\n\xe2\x96\xb6 https://t.co/d6DEdmu6ka\n\xf0\x9f\x93\x8d 19 Avenue Gordon Bennett, 63380 Pontaumur https://t.co/NtuZsGNbrX"""
3,1394415788195565578,2021-05-17 22:13:25,Clermont Ferrand,7,False,0,0,"b""\xf0\x9f\x92\x89 3 cr\xc3\xa9neaux disponibles\n\xf0\x9f\x93\x85 Aujourd'hui \xc3\xa0 16:15\n\xf0\x9f\x8f\xa5 Centre de vaccination COVID des Combrailles - Pontaumur (Moderna)\n\xe2\x96\xb6 https://t.co/d6DEdmu6ka\n\xf0\x9f\x93\x8d 19 Avenue Gordon Bennett, 63380 Pontaumur https://t.co/raWuCKWhue"""
4,1394409263284506626,2021-05-17 21:47:29,France.,389,False,0,0,b'@scofieldiego Ah ouais ?\nModerna et toi ?'


In [177]:
moderna_fr.loc[1,"Tweet Text"]

'b"@Sante_Gouv 1ere dose aujourd\'hui en moderna pour moi a part douleur a l\'\\xc3\\xa9paule tout vas bien :)"'

In [77]:
b"@Sante_Gouv 1ere dose aujourd'hui en moderna pour moi a part douleur a l'\xc3\xa9paule tout vas bien :)".decode("utf-8")

"@Sante_Gouv 1ere dose aujourd'hui en moderna pour moi a part douleur a l'épaule tout vas bien :)"

In [180]:
moderna_fr["Tweet Text"].map(lambda x: eval(x).decode("utf-8").replace('\n', ' '))

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      @BFMTV Moderna est quoi alors?
1                                                                                                                                                                                                                                                                                                                                                                                                                    @Sante_Gouv 1ere dose aujourd'hui en moderna pour moi a part douleur a l'épaule tout vas bien

**USA:**

In [194]:
# merging the files
joined_files = os.path.join("../data/tweets_us/us/", "tweets_us*.csv")
  
# A list of all joined files is returned
joined_list = glob.glob(joined_files)
  
# Finally, the files are joined
df = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)

In [195]:
df.shape

(50000, 9)